In [2]:
import os

In [121]:
def is_there_a_file(path: str) -> bool:
    return os.path.exists(path)

def check_file_extension(path: str, extension: str) -> bool:
    return os.path.splitext(path)[1] == extension

def valid_path_file(path: str)-> bool:
    return is_there_a_file(path) and check_file_extension(path, '.txt')

def get_all_files_by_extension(path: str, extension: str)-> list:
    return [file_name for file_name in os.listdir(path) if extension in file_name]

def read_file(path: str):
    all_data = []
    text = []
    with open(path, 'r') as file:
        for line in file:
            line = line.rstrip('\n').split(',')

            xn_and_yn = turn_all_elements_into_int(line[:8].copy())
            text_data = process_text_data(line[8:].copy())

            
            xn_and_yn.append(text_data)
            
            text.append(text_data)
            all_data.append(xn_and_yn)
        
        return all_data, text

def turn_all_elements_into_int(list_str: list)-> list:
    return list(map(lambda number: int(number), list_str))

def process_text_data(list_text: list)-> str:
    if len(list_text) > 1:
        list_text = ','.join(list_text)
    else:
        list_text = list_text[0]
        
    return list_text


In [122]:
file_path = './test_files/OCR1.txt'
directory_path = './test_files'

valid_path_file(file_path)
get_all_files_by_extension(directory_path,'.txt')
all, text = read_file(file_path)
all

[[119, 47, 367, 47, 367, 80, 119, 80, 'TAN WOON YANN'],
 [93, 161, 352, 161, 352, 183, 93, 183, 'MR D.T.Y. (JOHOR) SDN BHD'],
 [137, 185, 320, 185, 320, 204, 137, 204, '(CO.REG : 933109-X)'],
 [49, 201, 391, 201, 391, 227, 49, 227, 'LOT 1851-A & 1851-B, JALAN KPB 6,'],
 [60, 228, 382, 228, 382, 246, 60, 246, 'KAWASAN PERINDUSTRIAN BALAKONG,'],
 [70, 249, 384, 249, 384, 267, 70, 267, '43300 SERI KEMBANGAN, SELANGOR'],
 [116, 268, 331, 268, 331, 289, 116, 289, '(MR DIY TESCO TERBAU)'],
 [176, 290, 268, 290, 268, 308, 176, 308, '-INVOICE-'],
 [14, 328, 358, 328, 358, 351, 14, 351, 'CHOPPING BOARD 35.5X25.5CM 803M#'],
 [16, 349, 158, 349, 158, 372, 16, 372, 'EZ10HD05 - 24'],
 [20, 371, 95, 371, 95, 393, 20, 393, '8970669'],
 [249, 373, 259, 373, 259, 393, 249, 393, '1'],
 [266, 376, 282, 376, 282, 393, 266, 393, 'X'],
 [290, 374, 343, 374, 343, 395, 290, 395, '19.00'],
 [358, 375, 412, 375, 412, 396, 358, 396, '19.00'],
 [15, 394, 371, 394, 371, 417, 15, 417, 'AIR PRESSURE SPRAYER SX-575-1